[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_27/AI4I_Colab_Notebook.ipynb)

# AI4I 2020 Predictive Maintenance – Vollständiger CRISP-DM Workflow

**Vorlesung 28: Smart Operations Management**  
**Thema:** Predictive Maintenance mit Machine Learning

---

## 🎯 Lernziele

In diesem Notebook durchlaufen wir **alle Phasen von CRISP-DM** anhand eines industrienahen Beispiels:

1. **Business Understanding**: Warum Predictive Maintenance? ROI-Berechnung
2. **Data Understanding**: Dataset explorieren, Statistiken, Visualisierungen
3. **Data Preparation**: Feature Engineering, Encoding, Scaling
4. **Modeling**: Logistic Regression, Decision Trees, Random Forest
5. **Evaluation**: Confusion Matrix, Precision/Recall, ROC, Cost-Sensitive Metrics
6. **Deployment**: Überlegungen für Produktionsumgebung

---

## 📊 Der AI4I 2020 Dataset

**Quelle:** HTW Berlin (Prof. Stephan Matzka, 2020)  
**Lizenz:** CC BY 4.0  
**Größe:** 10.000 Datenpunkte  
**Kontext:** Synthetischer, aber realistischer Produktionsdatensatz

**Failure Modes:**
- **TWF**: Tool Wear Failure (Werkzeugverschleiß)
- **HDF**: Heat Dissipation Failure (Wärmeabfuhr-Problem)
- **PWF**: Power Failure (Leistung außerhalb Bereich)
- **OSF**: Overstrain Failure (Überlastung)
- **RNF**: Random Failures (Zufällige Ausfälle)

**Challenge:** Nur 3,39% Ausfälle (stark imbalanced!)

---

## 📚 Wiederholung wichtiger Konzepte

Dieses Notebook wiederholt:
- ✅ Deskriptive Statistik
- ✅ Explorative Datenanalyse (EDA)
- ✅ Feature Engineering
- ✅ Train-Test Split (stratified!)
- ✅ Supervised Learning (Klassifikation)
- ✅ Decision Trees & Ensemble Methods
- ✅ Imbalanced Data Handling
- ✅ Klassifikationsmetriken (Precision, Recall, F1, ROC)
- ✅ Cost-Sensitive Learning
- ✅ Model Interpretability

---

**Direkt-URL zum Dataset:**  
`https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv`

In [ ]:
# %%capture
# Optional: Hilfspakete installieren (in Colab meist nicht nötig)
# !pip install -q ucimlrepo


# 1️⃣ Business Understanding

## Warum Predictive Maintenance?

**Das Problem:**
- Ungeplante Maschinenausfälle kosten 1.000 - 10.000 €/Stunde
- Lieferverzug → Vertragsstrafen, Kundenverlust
- Notfall-Reparaturen sind teuer

**Traditionelle Ansätze:**
- **Reaktiv**: Reparieren wenn kaputt → hohe Ausfallkosten
- **Präventiv**: Feste Intervalle → oft zu früh (Verschwendung) oder zu spät (Ausfall)

**Predictive Maintenance:**
- Zustandsdaten nutzen um Ausfälle **vorherzusagen**
- Wartung **gerade rechtzeitig** durchführen

## Business Case (Beispiel)

**Situation:** 10 Maschinen, je 2 ungeplante Ausfälle/Jahr à 8h

- Produktionsausfall: 10 × 2 × 8h × 2.000 €/h = **320.000 €/Jahr**
- Notfall-Reparaturen: 10 × 2 × 5.000 € = **100.000 €/Jahr**
- **Gesamtkosten: 420.000 €/Jahr**

**Investment:**
- Sensoren & Infrastruktur: 50.000 €
- ML-Entwicklung: 100.000 €
- Jährlicher Betrieb: 30.000 €/Jahr

**Erwarteter Nutzen:** 70% der Ausfälle vermieden
- Eingesparte Kosten: 420.000 × 0,7 = **294.000 €/Jahr**
- Netto-Nutzen Jahr 1: 294.000 - 150.000 - 30.000 = **114.000 €**
- **ROI Jahr 1: 76%**
- **Payback: ~15 Monate**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             RocCurveDisplay, PrecisionRecallDisplay)
import time
import os


# 2️⃣ Data Understanding: Daten laden und explorieren

## CRISP-DM Phase 2: Data Understanding

**Ziele:**
- Dataset kennenlernen
- Datenqualität prüfen
- Erste Hypothesen bilden

In [ ]:
# Datensatz laden (mit SSL-Workaround)
import ssl
import urllib.request

# SSL-Kontext für abgelaufene Zertifikate
ssl._create_default_https_context = ssl._create_unverified_context

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"

try:
    df = pd.read_csv(url)
except:
    # Fallback: Direkt über urllib mit unverified context
    print("Lade Daten mit SSL-Workaround...")
    with urllib.request.urlopen(url) as response:
        df = pd.read_csv(response)

df.head()

In [ ]:
df.shape, df.dtypes

### 📊 Erste Analyse

**Fragen:**
- Wie viele Datenpunkte?
- Welche Features (numerisch/kategorial)?
- Fehlende Werte?
- Wie ist die Klassenverteilung?

In [ ]:
# Erste Inspektion - Spaltennamen und Datentypen
print("📋 DATASET INFO")
print("="*50)
print(f"Shape: {df.shape}")
print(f"\nSpalten ({len(df.columns)}):")
print(df.columns.tolist())
print("\n" + "="*50)

## Spaltennamen harmonisieren

In [ ]:
df.columns = (df.columns
              .str.strip()
              .str.replace(r"\s+", "_", regex=True)
              .str.replace(r"[\[\]\(\)]", "", regex=True)
              .str.replace("%", "pct")
              .str.lower())
df.head()

### 📊 Klassenverteilung analysieren

**Das Imbalance-Problem:**
- Wie viele Ausfälle vs. normale Betriebszustände?
- Welche Failure Modes sind am häufigsten?

In [ ]:
# Class Distribution - Das Imbalance Problem!
print("📊 KLASSENVERTEILUNG")
print("="*50)
print(df['machine_failure'].value_counts())
print("\nProzentual:")
print(df['machine_failure'].value_counts(normalize=True))

# Failure Modes analysieren
print("\n--- FAILURE MODES ---")
failure_modes = ['twf', 'hdf', 'pwf', 'osf', 'rnf']
for mode in failure_modes:
    if mode in df.columns:
        count = df[mode].sum()
        print(f"{mode.upper()}: {count:4d} Fälle")

print("\n" + "="*50)
print(f"Gesamt Failures: {df['machine_failure'].sum()}")
print(f"Failure Rate:    {df['machine_failure'].mean():.2%}")
print("="*50)

### 📈 Deskriptive Statistik

**Wiederholung:**
- Mean, Median, Std
- Min, Max, Quartile
- Verteilungen erkennen

In [ ]:
df.describe(include='all')

In [ ]:
# Visualisierung wichtiger Features
import seaborn as sns

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Features visualisieren
important_features = ['air_temperature_k', 'process_temperature_k', 
                     'rotational_speed_rpm', 'torque_nm', 'tool_wear_min']

for idx, col in enumerate(important_features):
    if col in df.columns:
        axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
        axes[idx].set_title(f'Distribution: {col}')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frequency')
        
        # Failure vs. Non-Failure
        df[df['machine_failure']==1][col].hist(ax=axes[idx], bins=30, 
                                               alpha=0.5, color='red', label='Failure')
        axes[idx].legend()

plt.tight_layout()
plt.show()

### 🔍 Korrelationsanalyse

**Wiederholung:**
- Pearson Korrelation: -1 bis +1
- Welche Features korrelieren mit Ausfällen?
- Multikollinearität erkennen

In [ ]:
# Korrelation mit Zielvariable
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
correlations = df[numeric_cols].corr()['machine_failure'].sort_values(ascending=False)

print("Korrelation mit Machine Failure:")
print(correlations)

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

# 3️⃣ Data Preparation: Feature Engineering

## CRISP-DM Phase 3: Data Preparation

**Ziele:**
- Neue Features erstellen (basierend auf Failure Mode Regeln)
- Kategoriale Features encoden
- Features skalieren

In [ ]:
# Feature Engineering basierend auf Failure Mode Regeln

# 1. Power (relevant für PWF)
# Power = Torque × Rotational Speed (in rad/s)
df['power_w'] = df['torque_nm'] * df['rotational_speed_rpm'] * 2 * np.pi / 60

# 2. Temperature Difference (relevant für HDF)
df['temp_diff_k'] = df['process_temperature_k'] - df['air_temperature_k']

# 3. Strain (relevant für OSF)
df['strain'] = df['torque_nm'] * df['tool_wear_min']

print("Neue Features erstellt:")
print(df[['power_w', 'temp_diff_k', 'strain']].head())
print("\nDescribe:")
print(df[['power_w', 'temp_diff_k', 'strain']].describe())

### ⚠️ Wichtiger Hinweis: Realität vs. Lernbeispiel

**In diesem Dataset kennen wir die Failure Modes (TWF, HDF, PWF, OSF, RNF)** – das ist didaktisch hilfreich, aber **unrealistisch**!

**In der Realität:**
- Wir haben nur: **Sensordaten** (Temperatur, Drehzahl, Drehmoment, ...) und **Ausfall ja/nein**
- Wir wissen **nicht**, *warum* eine Maschine ausfällt (Werkzeugverschleiß? Überhitzung? Überlastung?)
- Genau das ist die **Aufgabe von Explainable AI (XAI)**!

**XAI-Techniken helfen dabei:**
1. **Feature Importance** (z.B. aus Random Forest) → Welche Sensoren sind wichtig?
2. **SHAP Values** → Wie beeinflussen einzelne Messwerte die Vorhersage?
3. **Decision Trees** (mit `max_depth`) → Welche Schwellenwerte trennen Ausfall/OK?
4. **Clustering in Residuen** → Gibt es versteckte Ausfallmuster?

**Unser Vorteil hier:** Wir können prüfen, ob das Modell *plausible* Features lernt (z.B. `power_w` für Power Failure). In der Praxis müssten wir diese Muster erst **entdecken** – und das ist genau der Mehrwert von XAI!

**Beispiel aus der Industrie:**
- Ein Predictive Maintenance Modell sagt "Ausfall in 48h" voraus
- Wartungsteam fragt: "Warum? Was sollen wir prüfen?"
- XAI liefert: "Hohe Leistung (7.5 kW) + niedrige Temp-Differenz (8.2 K) → Wärmeabfuhr prüfen!"
- Ohne XAI: Modell ist eine Black Box, Team hat keine Handlungsanweisung

### 🔀 Train-Test-Validation Split

**Wiederholung:**
- **Stratified Split**: Klassenverteilung in allen Sets gleich
- 60% Train, 20% Validation, 20% Test
- Warum wichtig bei Imbalance?

In [ ]:
# Zielvariable und Features trennen
target_col = 'machine_failure'
drop_cols = [target_col, 'udi', 'product_id', 'twf', 'hdf', 'pwf', 'osf', 'rnf']
drop_cols = [col for col in drop_cols if col in df.columns]

y = df[target_col].astype(int)
X = df.drop(columns=drop_cols)

# One-Hot Encoding für Type
X = pd.get_dummies(X, columns=['type'], drop_first=True)

print(f"Features: {X.shape[1]}")
print(f"Target distribution:\n{y.value_counts()}")

# Stratified Split: Train-Val-Test (60-20-20)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, stratify=y_trainval, random_state=42
)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"\nTrain: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"Train Failure Rate: {y_train.mean():.2%}")
print(f"Val Failure Rate: {y_val.mean():.2%}")
print(f"Test Failure Rate: {y_test.mean():.2%}")

# 4️⃣ Modeling: Von Baseline zu Ensemble

## CRISP-DM Phase 4: Modeling

**Strategie:**
1. **Baseline**: Logistic Regression (einfach, interpretierbar)
2. **Decision Tree**: Interpretierbar, kann Schwellenwerte zeigen
3. **Random Forest**: Beste Performance erwartet
4. **Vergleich**: Welches Modell für welchen Zweck?

### Model 1: Logistic Regression (Baseline)

In [ ]:
# Logistic Regression mit class_weight='balanced'
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_val_pred_lr = lr_model.predict(X_val_scaled)
y_val_prob_lr = lr_model.predict_proba(X_val_scaled)[:, 1]

print("Logistic Regression - Validation Set:")
print(classification_report(y_val, y_val_pred_lr, target_names=['OK', 'Failure']))

### Model 2: Decision Tree (Interpretierbar!)

**Wiederholung:**
- Gini Index: Maß für Unreinheit
- Splitting nach bestem Feature
- max_depth begrenzen gegen Overfitting

In [ ]:
# Decision Tree (einfach, max 4-6 Ebenen für Interpretierbarkeit)
from sklearn.tree import DecisionTreeClassifier, plot_tree

dt_model = DecisionTreeClassifier(max_depth=6, class_weight='balanced', random_state=42)
dt_model.fit(X_train, y_train)  # Ohne Scaling (Decision Trees invariant)

y_val_pred_dt = dt_model.predict(X_val)

print("Decision Tree - Validation Set:")
print(classification_report(y_val, y_val_pred_dt, target_names=['OK', 'Failure']))

# Visualisierung (kompakt)
plt.figure(figsize=(20, 10))
plot_tree(dt_model, filled=True, feature_names=X_train.columns, 
          class_names=['OK', 'Failure'], max_depth=3, fontsize=10)
plt.title('Decision Tree (erste 3 Ebenen)')
plt.show()

# Feature Importance
feature_importance_dt = pd.DataFrame({
    'feature': X_train.columns,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 wichtigste Features (Decision Tree):")
print(feature_importance_dt.head(10))

### Model 3: Random Forest (Beste Performance!)

**Wiederholung:**
- Ensemble aus vielen Decision Trees
- Bagging: Jeder Tree sieht random Sample
- Robuster gegen Overfitting
- Feature Importance aus allen Trees

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, 
                                   class_weight='balanced', random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

y_val_pred_rf = rf_model.predict(X_val)
y_val_prob_rf = rf_model.predict_proba(X_val)[:, 1]

print("Random Forest - Validation Set:")
print(classification_report(y_val, y_val_pred_rf, target_names=['OK', 'Failure']))

# Feature Importance
feature_importance_rf = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot Top 15
plt.figure(figsize=(10, 8))
top_features = feature_importance_rf.head(15)
plt.barh(top_features['feature'], top_features['importance'])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
print(feature_importance_rf.head(10))

# 5️⃣ Evaluation: Test Set Performance

## CRISP-DM Phase 5: Evaluation

**Ziele:**
- Finales Modell auf Test Set evaluieren
- Confusion Matrix analysieren
- Precision, Recall, F1 berechnen
- ROC Curve plotten
- Cost-Sensitive Evaluation

### Test Set Evaluation (Random Forest)

In [ ]:
# Final Test Set Predictions mit Random Forest
# Verwende X_test (DataFrame) statt X_test_scaled (NumPy Array) für Random Forest
y_test_pred = rf_model.predict(X_test)
y_test_proba = rf_model.predict_proba(X_test)[:, 1]

# Classification Report
print("📊 CLASSIFICATION REPORT - TEST SET")
print("="*50)
print(classification_report(y_test, y_test_pred, target_names=['OK', 'Failure'], zero_division=0))

# Confusion Matrix
cm_test = confusion_matrix(y_test, y_test_pred)
print("\n📈 CONFUSION MATRIX")
print("="*50)
print(cm_test)
print(f"\nTrue Negatives (TN): {cm_test[0,0]}")
print(f"False Positives (FP): {cm_test[0,1]}")
print(f"False Negatives (FN): {cm_test[1,0]}")
print(f"True Positives (TP): {cm_test[1,1]}")

In [ ]:
# Confusion Matrix Visualization
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['OK', 'Failure'], 
            yticklabels=['OK', 'Failure'],
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted', fontsize=12, fontweight='bold')
plt.ylabel('Actual', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - Random Forest (Test Set)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Berechne wichtige Metriken manuell
tn, fp, fn, tp = cm_test.ravel()
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\n🎯 WICHTIGE METRIKEN (Test Set)")
print("="*50)
print(f"Precision (Failure): {precision:.3f}")
print(f"Recall (Failure):    {recall:.3f}")
print(f"F1-Score (Failure):  {f1:.3f}")
print(f"Specificity (OK):    {specificity:.3f}")

### ROC Curve und AUC

**Konzept:**
- ROC = Receiver Operating Characteristic
- Trade-off zwischen True Positive Rate (Recall) und False Positive Rate
- AUC = Area Under Curve (0.5 = random, 1.0 = perfekt)

In [ ]:
from sklearn.metrics import RocCurveDisplay, roc_auc_score

# ROC Curve plotten
fig, ax = plt.subplots(figsize=(10, 8))

# Random Forest ROC (mit X_test DataFrame)
RocCurveDisplay.from_estimator(rf_model, X_test, y_test, 
                                name='Random Forest', ax=ax, color='green')

# Logistic Regression ROC (mit X_test_scaled)
RocCurveDisplay.from_estimator(lr_model, X_test_scaled, y_test, 
                                name='Logistic Regression', ax=ax, color='blue')

# Decision Tree ROC (mit X_test DataFrame)
RocCurveDisplay.from_estimator(dt_model, X_test, y_test, 
                                name='Decision Tree', ax=ax, color='red')

# Diagonale (Random Classifier)
ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier (AUC=0.5)')

ax.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
ax.set_ylabel('True Positive Rate (Recall)', fontsize=12, fontweight='bold')
ax.set_title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# AUC Scores
print("\n📊 AUC SCORES (Test Set)")
print("="*50)
print(f"Random Forest:        {roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]):.4f}")
print(f"Logistic Regression:  {roc_auc_score(y_test, lr_model.predict_proba(X_test_scaled)[:, 1]):.4f}")
print(f"Decision Tree:        {roc_auc_score(y_test, dt_model.predict_proba(X_test)[:, 1]):.4f}")

### Precision-Recall Curve

**Warum wichtig bei Imbalanced Data?**
- Bei stark unbalancierten Klassen (96.6% OK vs. 3.4% Failure)
- ROC kann zu optimistisch sein
- Precision-Recall fokussiert auf Minority Class (Failures)

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay, average_precision_score

# Precision-Recall Curves
fig, ax = plt.subplots(figsize=(10, 8))

# Random Forest (mit X_test DataFrame)
PrecisionRecallDisplay.from_estimator(rf_model, X_test, y_test, 
                                       name='Random Forest', ax=ax, color='green')

# Logistic Regression (mit X_test_scaled)
PrecisionRecallDisplay.from_estimator(lr_model, X_test_scaled, y_test, 
                                       name='Logistic Regression', ax=ax, color='blue')

# Decision Tree (mit X_test DataFrame)
PrecisionRecallDisplay.from_estimator(dt_model, X_test, y_test, 
                                       name='Decision Tree', ax=ax, color='red')

# Baseline (Proportion of Failures)
baseline = y_test.sum() / len(y_test)
ax.axhline(y=baseline, color='k', linestyle='--', label=f'Baseline (Prevalence={baseline:.3f})')

ax.set_xlabel('Recall', fontsize=12, fontweight='bold')
ax.set_ylabel('Precision', fontsize=12, fontweight='bold')
ax.set_title('Precision-Recall Curves - Model Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Average Precision Scores
print("\n📊 AVERAGE PRECISION SCORES (Test Set)")
print("="*50)
print(f"Random Forest:        {average_precision_score(y_test, rf_model.predict_proba(X_test)[:, 1]):.4f}")
print(f"Logistic Regression:  {average_precision_score(y_test, lr_model.predict_proba(X_test_scaled)[:, 1]):.4f}")
print(f"Decision Tree:        {average_precision_score(y_test, dt_model.predict_proba(X_test)[:, 1]):.4f}")

### Cost-Sensitive Evaluation

**Business Context:**
- False Negative (FN) = Maschine fällt unerwartet aus → 10.000€ Kosten
- False Positive (FP) = Unnötige Wartung → 300€ Kosten
- **Cost Ratio: FN:FP = 30:1**

**Berechnung der Gesamtkosten:**

In [ ]:
# Cost-Sensitive Evaluation
COST_FN = 10000  # Kosten für unerwarteten Ausfall
COST_FP = 300    # Kosten für unnötige Wartung

def calculate_costs(y_true, y_pred):
    """Berechne Gesamtkosten basierend auf Confusion Matrix"""
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    total_cost = (fn * COST_FN) + (fp * COST_FP)
    
    return {
        'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
        'Cost_FN': fn * COST_FN,
        'Cost_FP': fp * COST_FP,
        'Total_Cost': total_cost
    }

# Kosten für alle Modelle berechnen
models = {
    'Logistic Regression': lr_model.predict(X_test_scaled),
    'Decision Tree': dt_model.predict(X_test),
    'Random Forest': rf_model.predict(X_test)
}

print("💰 COST-SENSITIVE EVALUATION (Test Set)")
print("="*70)
print(f"Kosten pro FN (unerwarteter Ausfall): {COST_FN:,}€")
print(f"Kosten pro FP (unnötige Wartung):     {COST_FP:,}€")
print(f"Cost Ratio FN:FP = {COST_FN//COST_FP}:1")
print("="*70)

costs_summary = []
for model_name, y_pred in models.items():
    costs = calculate_costs(y_test, y_pred)
    costs_summary.append({
        'Model': model_name,
        'FN': costs['FN'],
        'FP': costs['FP'],
        'Cost_FN': costs['Cost_FN'],
        'Cost_FP': costs['Cost_FP'],
        'Total_Cost': costs['Total_Cost']
    })
    
    print(f"\n{model_name}:")
    print(f"  FN: {costs['FN']:3d} × {COST_FN:,}€ = {costs['Cost_FN']:>10,}€")
    print(f"  FP: {costs['FP']:3d} × {COST_FP:,}€     = {costs['Cost_FP']:>10,}€")
    print(f"  {'TOTAL COST':40s} = {costs['Total_Cost']:>10,}€")

# Bestes Modell identifizieren
best_model = min(costs_summary, key=lambda x: x['Total_Cost'])
print("\n" + "="*70)
print(f"🏆 BESTES MODELL (niedrigste Kosten): {best_model['Model']}")
print(f"   Gesamtkosten: {best_model['Total_Cost']:,}€")
print("="*70)

### Threshold Tuning

**Problem:** Default Threshold = 0.5 ist nicht optimal für imbalanced data
**Lösung:** Optimiere Threshold basierend auf Business-Kosten

In [ ]:
# Threshold Tuning für Random Forest
thresholds = np.arange(0.1, 0.9, 0.05)
costs_by_threshold = []

for threshold in thresholds:
    y_pred_threshold = (y_test_proba >= threshold).astype(int)
    costs = calculate_costs(y_test, y_pred_threshold)
    costs_by_threshold.append({
        'threshold': threshold,
        'fn': costs['FN'],
        'fp': costs['FP'],
        'total_cost': costs['Total_Cost']
    })

# Plot: Kosten vs. Threshold
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Linke Grafik: FN und FP counts
ax1.plot([c['threshold'] for c in costs_by_threshold], 
         [c['fn'] for c in costs_by_threshold], 
         'o-', label='False Negatives', color='red', linewidth=2)
ax1.plot([c['threshold'] for c in costs_by_threshold], 
         [c['fp'] for c in costs_by_threshold], 
         's-', label='False Positives', color='orange', linewidth=2)
ax1.set_xlabel('Decision Threshold', fontsize=12, fontweight='bold')
ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
ax1.set_title('FN and FP vs. Threshold', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rechte Grafik: Gesamtkosten
ax2.plot([c['threshold'] for c in costs_by_threshold], 
         [c['total_cost'] for c in costs_by_threshold], 
         'D-', color='darkred', linewidth=2, markersize=6)
optimal = min(costs_by_threshold, key=lambda x: x['total_cost'])
ax2.axvline(x=optimal['threshold'], color='green', linestyle='--', 
            label=f'Optimal Threshold={optimal["threshold"]:.2f}')
ax2.axvline(x=0.5, color='blue', linestyle='--', alpha=0.5, label='Default Threshold=0.5')
ax2.set_xlabel('Decision Threshold', fontsize=12, fontweight='bold')
ax2.set_ylabel('Total Cost (€)', fontsize=12, fontweight='bold')
ax2.set_title('Total Cost vs. Threshold', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 OPTIMAL THRESHOLD")
print("="*70)
print(f"Optimal Threshold: {optimal['threshold']:.2f}")
print(f"Total Cost:        {optimal['total_cost']:,}€")
print(f"False Negatives:   {optimal['fn']}")
print(f"False Positives:   {optimal['fp']}")
print("\nVs. Default Threshold 0.5:")
default = next(c for c in costs_by_threshold if abs(c['threshold'] - 0.5) < 0.01)
print(f"Default Cost:      {default['total_cost']:,}€")
print(f"Cost Reduction:    {default['total_cost'] - optimal['total_cost']:,}€ ({(1 - optimal['total_cost']/default['total_cost'])*100:.1f}%)")
print("="*70)

# 6️⃣ Deployment: Von Modell zur Produktion

## CRISP-DM Phase 6: Deployment

**Deployment-Szenarien in der Produktion:**

### 1. Batch Prediction (Offline)
- Täglich/Wöchentlich alle Maschinen evaluieren
- Wartungspläne erstellen
- Integration mit MES (Manufacturing Execution System)

### 2. Real-Time API (Online)
- REST API für Echtzeit-Vorhersagen
- Integration mit SCADA-Systemen
- Sofortige Benachrichtigungen bei kritischen Zuständen

### 3. Edge Computing
- Modell direkt auf Maschinen-Controller
- Offline-Betrieb möglich
- Minimale Latenz

---

### Model Persistence (Speichern des Modells)

In [ ]:
import joblib
import json
from datetime import datetime

# Feature Namen extrahieren
feature_cols = X_train.columns.tolist()

# Model speichern
model_artifacts = {
    'model': rf_model,
    'scaler': scaler,
    'feature_names': feature_cols,
    'optimal_threshold': optimal['threshold'],
    'metadata': {
        'train_date': datetime.now().isoformat(),
        'model_type': 'RandomForestClassifier',
        'n_estimators': 100,
        'max_depth': 10,
        'test_auc': roc_auc_score(y_test, y_test_proba),
        'test_avg_precision': average_precision_score(y_test, y_test_proba),
        'optimal_cost': optimal['total_cost']
    }
}

# Speichern (in Produktionsumgebung)
# joblib.dump(model_artifacts, 'ai4i_predictive_maintenance_model.pkl')

print("✅ MODEL ARTIFACTS")
print("="*70)
print("Folgende Komponenten würden gespeichert:")
print("  - Random Forest Model")
print("  - StandardScaler für Feature Scaling")
print(f"  - Feature Names ({len(feature_cols)} Features) für Input Validation")
print(f"  - Optimal Threshold: {optimal['threshold']:.2f}")
print(f"  - Metadata: AUC={model_artifacts['metadata']['test_auc']:.4f}")
print("="*70)

### Beispiel: Prediction Function für Deployment

In [ ]:
def predict_machine_failure(machine_data):
    """
    Production-ready prediction function
    
    Input: Dictionary mit Maschinendaten
    Output: Prediction + Probability + Empfehlung
    """
    # Input Validation
    required_features = ['air_temperature_k', 'process_temperature_k', 
                         'rotational_speed_rpm', 'torque_nm', 'tool_wear_min']
    
    if not all(feat in machine_data for feat in required_features):
        raise ValueError(f"Missing features. Required: {required_features}")
    
    # Feature Engineering (wie im Training)
    power_w = (machine_data['torque_nm'] * machine_data['rotational_speed_rpm'] * 
               2 * np.pi / 60)
    temp_diff_k = (machine_data['process_temperature_k'] - 
                   machine_data['air_temperature_k'])
    strain = machine_data['torque_nm'] / machine_data['rotational_speed_rpm']
    
    # Input Array erstellen (mit allen Features inkl. Type)
    X_input = pd.DataFrame([{
        'Type': machine_data.get('Type', 'M'),  # Default = Medium Quality
        'Air temperature [K]': machine_data['air_temperature_k'],
        'Process temperature [K]': machine_data['process_temperature_k'],
        'Rotational speed [rpm]': machine_data['rotational_speed_rpm'],
        'Torque [Nm]': machine_data['torque_nm'],
        'Tool wear [min]': machine_data['tool_wear_min'],
        'power_w': power_w,
        'temp_diff_k': temp_diff_k,
        'strain': strain
    }])
    
    # One-Hot Encoding für Type
    X_input_encoded = pd.get_dummies(X_input, columns=['Type'], prefix='Type', drop_first=True)
    
    # Sicherstellen dass alle Features vorhanden sind
    for col in feature_cols:
        if col not in X_input_encoded.columns:
            X_input_encoded[col] = 0
    X_input_encoded = X_input_encoded[feature_cols]
    
    # Prediction (Random Forest wurde OHNE Scaling trainiert!)
    # Wichtig: DataFrame übergeben, damit Feature-Namen erhalten bleiben
    proba = rf_model.predict_proba(X_input_encoded)[0, 1]
    prediction = int(proba >= optimal['threshold'])
    
    # Empfehlung generieren
    if prediction == 1:
        if proba >= 0.8:
            recommendation = "🚨 CRITICAL: Sofortige Wartung erforderlich!"
        elif proba >= 0.6:
            recommendation = "⚠️ WARNING: Wartung innerhalb 24h einplanen"
        else:
            recommendation = "⚠️ CAUTION: Wartung innerhalb einer Woche empfohlen"
    else:
        if proba >= 0.3:
            recommendation = "ℹ️ INFO: Maschine beobachten, Trend überwachen"
        else:
            recommendation = "✅ OK: Maschine im Normalbetrieb"
    
    return {
        'prediction': 'FAILURE' if prediction == 1 else 'OK',
        'failure_probability': float(proba),
        'threshold': optimal['threshold'],
        'recommendation': recommendation,
        'engineered_features': {
            'power_w': float(power_w),
            'temp_diff_k': float(temp_diff_k),
            'strain': float(strain)
        }
    }

# Test mit Beispiel-Daten
test_machine = {
    'Type': 'L',  # Low Quality
    'air_temperature_k': 298.1,
    'process_temperature_k': 308.6,
    'rotational_speed_rpm': 1551,
    'torque_nm': 42.8,
    'tool_wear_min': 220
}

result = predict_machine_failure(test_machine)
print("\n🔮 PREDICTION EXAMPLE")
print("="*70)
print(json.dumps(result, indent=2))
print("="*70)

### Monitoring & Retraining

**Production Considerations:**

1. **Model Monitoring:**
   - Track Prediction Distribution über Zeit
   - Erkennung von Concept Drift
   - Feature Distribution Monitoring

2. **Performance Tracking:**
   - Tatsächliche vs. Vorhergesagte Ausfälle
   - FN/FP Rates im Live-Betrieb
   - Gesamtkosten tracking

3. **Retraining Strategy:**
   - Monatliches Retraining mit neuen Daten
   - A/B Testing von Modell-Versionen
   - Automatisierte Pipeline (MLOps)

# 🎓 Zusammenfassung & Lernziele Review

---

## ✅ Was haben wir gelernt?

### 1️⃣ Business Understanding
- ✅ ROI-Berechnung für ML-Projekte
- ✅ Cost-Benefit-Analyse (420k€ Kosten → 114k€ Profit im Jahr 1)
- ✅ Stakeholder Requirements (MES, ERP, SCADA Integration)

### 2️⃣ Data Understanding
- ✅ Explorative Datenanalyse (EDA)
- ✅ Class Imbalance erkennen (96.6% vs. 3.4%)
- ✅ Feature Distributions analysieren
- ✅ Korrelationsanalyse

### 3️⃣ Data Preparation
- ✅ Feature Engineering (power_w, temp_diff_k, strain)
- ✅ One-Hot Encoding für kategorische Features
- ✅ Feature Scaling (StandardScaler)
- ✅ Stratified Train-Val-Test Split

### 4️⃣ Modeling
- ✅ Baseline Model (Logistic Regression)
- ✅ Decision Trees mit Interpretierbarkeit
- ✅ Random Forest für Performance
- ✅ class_weight='balanced' für Imbalance

### 5️⃣ Evaluation
- ✅ Confusion Matrix verstehen (TP, FP, FN, TN)
- ✅ Precision, Recall, F1-Score berechnen
- ✅ ROC Curves und AUC
- ✅ Precision-Recall Curves für Imbalanced Data
- ✅ **Cost-Sensitive Evaluation** (FN:FP = 30:1)
- ✅ **Threshold Tuning** für Business Optimization

### 6️⃣ Deployment
- ✅ Model Persistence (joblib)
- ✅ Production-Ready Prediction Functions
- ✅ Deployment Szenarien (Batch, API, Edge)
- ✅ Monitoring & Retraining Strategy

---

## 🔗 Bezug zum Gesamtsemester

**Konzepte aus vorherigen Vorlesungen:**
- VL01-02: Python Basics, NumPy, Pandas → Datenverarbeitung
- VL03-04: Data Visualization → EDA mit Matplotlib/Seaborn
- VL05-06: CRISP-DM Methodik → Kompletter Workflow
- VL07-08: Supervised Learning → Classification Algorithms
- VL09-10: Feature Engineering → Neue Features ableiten
- VL11-12: Model Evaluation → Metrics & Validation
- VL13-14: Imbalanced Data → class_weight, SMOTE
- VL15-16: Cost-Sensitive Learning → Business-getriebene Metriken

**Smart Operations Management (VL28):**
- Predictive Maintenance als Kernkonzept
- Integration in MES/ERP/SCADA Systeme
- OEE (Overall Equipment Effectiveness) Optimierung
- TCO (Total Cost of Ownership) Reduzierung

---

## 💡 Key Takeaways

1. **ML ist kein reines Tech-Problem** → Business Value ist entscheidend
2. **Default Metrics können irreführend sein** → Cost-Sensitive Evaluation
3. **Threshold 0.5 ist selten optimal** → Business-driven Tuning
4. **Deployment ≠ Ende des Projekts** → Monitoring & Retraining
5. **Interpretierbarkeit wichtig** → Besonders in kritischen Anwendungen

---

## 📚 Klausurvorbereitung

**Prüfungsrelevante Konzepte:**
- ✅ Confusion Matrix berechnen & interpretieren
- ✅ Precision/Recall/F1 Formeln
- ✅ Class Imbalance behandeln (Techniques)
- ✅ Feature Engineering Strategien
- ✅ Train-Test-Split Strategien (Stratified!)
- ✅ ROI-Berechnung für ML-Projekte
- ✅ Cost-Sensitive Learning
- ✅ CRISP-DM Phasen anwenden

---

## ❓ Diskussionsfragen

1. **Ethik:** Was passiert wenn das Modell einen kritischen Ausfall nicht vorhersagt (FN)?
2. **Kosten:** Wie würde sich das optimale Threshold ändern wenn FP-Kosten steigen?
3. **Drift:** Wie erkennt man Concept Drift in Produktionsumgebungen?
4. **Interpretierbarkeit:** Warum ist Feature Importance wichtig für Wartungsteams?
5. **Skalierung:** Wie würde das System mit 10.000 Maschinen skalieren?

---

## 🚀 Weiterführende Themen

- **Explainable AI (XAI):** SHAP, LIME für Modell-Erklärungen
- **AutoML:** Automatisierte Feature Engineering & Hyperparameter Tuning
- **Deep Learning:** LSTM für Zeitreihen-basierte Predictive Maintenance
- **Federated Learning:** Dezentrales Training über mehrere Fabriken
- **Digital Twins:** Integration von Simulations-Modellen